卷积神经网络是深度学习中最经典的模型之一。它巧妙地利用很少的权重却达到了全连接网络实现不了的效果。
# 全连接网络的局限性
在mnist多层分类中，仅使用一个28x28像素的小图片数据集就完成了分类任务。但在实际应用中要处理的图片像素一般都是1024，甚至更大。如果只有两个隐藏层，每层各用了256个节点，则MNIST数据集所需要的参数是(28x28x256+256x256+256x10)个w,再加上(256+ 256 +10)个b。

1. 图像变大导致色彩数变多，不好解决
如果换为1000像素，仅一层就需要1000x1000x256$ \approx $2亿个W。这只是灰度图，如果是RGB的真彩色图，再乘以3约等于6亿。如果再加上更多层，需要的学习参数将是非常多的，不仅消耗大量的内存，同时也需要大量的运算。

2. 不便处理高维数据
对于比较复杂的高维数据，如果按照全连接的方法，则只能通过增加节点、增加层数的方式来解决。而增加节点会引起参数过多的问题。因为由于隐藏层神经网络使用的是Sigmoid或Tanh激活函数，其反向传播的有效层数也只能在4~6层左右。所以，层数再多只会使反向传播的修正值越来越小，网络无法训练。

而卷积网络使用了**参数共享的方式**，换了一个角度来解决问题，不仅在准确率上大大提升，也把参数维度降了下来。

# 理解卷积神经网络
卷积网络避免了对参数的过度依赖，相比全连接神经网络，能更好地识别高维数据。
![sobel算子](imgs/17_sobel.png)上图为sobel算子对图片处理后的效果，它可以把图片的轮廓显示出来。它其实是一个很简单的矩阵计算。![](imgs/17_matrix_cal.png)图中5x5矩阵可以理解为图（小女孩原图），经过卷及操作后，变为轮廓图。
整个过程步骤如下：
1. 在外面补了一圈0，这个过程叫做padding，目的是为了变换后生成同样大小的矩阵。
2. 将3x3的矩阵中的每个元素分别与3x3矩阵对应位置上的元素相乘，然后再相加，这样得到的值作为图b的第一个元素。
3. 中间的3x3矩阵就是sobel算子。
4. 把图a中左上角的3x3矩阵向右移动一个格子，这可以理解为步长为1.
5. 将图a矩阵中的每个元素分别与中间的3x3矩阵对应位置上的元素相乘然后进行加和运算，算出的值填到图b的第二个元素里。
6. 一直重复上述操作，直到将图b中的值都填满，整个过程就叫做卷积。 

sobel可以理解为卷积神经网络里的卷积核（也可以叫做“滤波器”，filter），它里面的值也可以理解为权重w。在sobel中，这些w是固定的，就相当于一个训练好的模型，只要通过里面的值变换后的图片，就会产生具有轮廓的效果。这个变换后的图片，在卷积神经网络里称为**feature map**。

> 新生成的图片里面的每个像素值并不能保证在0~256之间。对于在区间外的像素点会导致灰度图无法显示